In [1]:
import pandas as pd
import numpy as np

In [2]:
def get_rates(hometeam,awayteam,dataset,num_games=10,method=1):
    '''
    Function to get transition rates for hometeam and awayteam scoring respectively.
    Inputs:
    hometeam - name of home team as in dataset
    awayteam - name of away team as in dataset
    dataset - dataset of historical results. This function expects the dataset input to have the following columns:
        Date - of the given historical fixture
        HomeTeam - name of the home team in a given fixture
        HomeTeamGoals - number of goals scored by HomeTeam in a given fixture
        AwayTeam -name of the away team in a given fixture
        AwayTeamGoals - number of goals scored in a given fixture
        Additionally, dataset must be sorted by descending Date (i.e. most recent historical fixtures occur first)
    num_games - number of historical figures to average over
    method - method for calculating the transition rates:
        1. Average of goals scored by team in the last num_games and goals conceded by their opponent in the last num_games
    '''
    num_games_home=min(len(dataset.Date[(dataset.HomeTeam==hometeam)|(dataset.AwayTeam==hometeam)]),num_games)
    dates_home=dataset.Date[(dataset.HomeTeam==hometeam)|(dataset.AwayTeam==hometeam)].reset_index(drop=True)
    earliest_date_home=dates_home[num_games_home-1]
    home_goals_for=np.sum(dataset.HomeTeamGoals[dataset.HomeTeam==hometeam][dataset.Date>=earliest_date_home])+ np.sum(dataset.AwayTeamGoals[dataset.AwayTeam==hometeam][dataset.Date>=earliest_date_home])
    home_goals_against=np.sum(dataset.AwayTeamGoals[dataset.HomeTeam==hometeam][dataset.Date>=earliest_date_home])+ np.sum(dataset.HomeTeamGoals[dataset.AwayTeam==hometeam][dataset.Date>=earliest_date_home])
    
    num_games_away=min(len(dataset.Date[(dataset.AwayTeam==awayteam)|(dataset.HomeTeam==awayteam)]),num_games)
    dates_away=dataset.Date[(dataset.AwayTeam==awayteam)|(dataset.HomeTeam==awayteam)].reset_index(drop=True)
    earliest_date_away=dates_away[num_games_away-1]
    away_goals_for=np.sum(dataset.AwayTeamGoals[dataset.AwayTeam==awayteam][dataset.Date>=earliest_date_away])+ np.sum(dataset.HomeTeamGoals[dataset.HomeTeam==awayteam][dataset.Date>=earliest_date_away])
    away_goals_against=np.sum(dataset.AwayTeamGoals[dataset.HomeTeam==awayteam][dataset.Date>=earliest_date_away])+ np.sum(dataset.HomeTeamGoals[dataset.AwayTeam==awayteam][dataset.Date>=earliest_date_away])
    
    if method==1:
        rate_home=(home_goals_for/num_games_home + away_goals_against/num_games_away)/2
        rate_away=(away_goals_for/num_games_away + home_goals_against/num_games_home)/2
    
    return [rate_home,rate_away]
        

In [3]:
def gillespie(rates,rseed=None):
    score=[0,0]
    if rseed is not None:
        np.random.set_state(rseed)
    t=0
    while t<1:
        dt=np.random.exponential(scale=1/sum(rates))
        whoscored=np.random.choice([0,1],p=rates/sum(rates))
        score[whoscored]+=1
        t+=dt
    return score

In [4]:
pl_results=pd.read_csv(r"data\2022-23 PL\2023_matchday_results.csv")
pl_results.head()

rename_cols_dict={'fixture.date':'Date','teams.home.name':'HomeTeam','teams.away.name':'AwayTeam','goals.home':'HomeTeamGoals','goals.away':'AwayTeamGoals'}
results=pl_results[rename_cols_dict.keys()].rename(columns=rename_cols_dict).sort_values(by='Date',ascending=False)

In [5]:
def predict_result(hometeam,awayteam,dataset,num_games=10,method=1,num_simulations=1000):
    rates=get_rates(hometeam,awayteam,dataset,num_games=num_games,method=method)
    res=[]
    for i in range(num_simulations):
        res.append(gillespie(rates))
    res=np.array(res)
    n_home_wins=len(res[res[:,0]>res[:,1]])
    n_away_wins=len(res[res[:,0]<res[:,1]])
    n_draws=num_simulations-n_home_wins-n_away_wins
    p_home_win, p_away_win, p_draw = n_home_wins/num_simulations, n_away_wins/num_simulations, n_draws/num_simulations
    return {'ProbHomeWin':p_home_win,'ProbAwayWin':p_away_win,'ProbDraw':p_draw,'FullSimulatedResults':res}

In [6]:
predict_result('Chelsea','Arsenal',results)

{'ProbHomeWin': 0.233,
 'ProbAwayWin': 0.592,
 'ProbDraw': 0.175,
 'FullSimulatedResults': array([[2, 3],
        [2, 3],
        [0, 6],
        ...,
        [1, 6],
        [4, 1],
        [2, 3]])}

In [10]:
teams=results.HomeTeam.unique()
print(teams, len(teams))

['Southampton' 'Manchester United' 'Leicester' 'Leeds' 'Everton'
 'Crystal Palace' 'Chelsea' 'Brentford' 'Aston Villa' 'Arsenal' 'Brighton'
 'Newcastle' 'Manchester City' 'West Ham' 'Nottingham Forest' 'Liverpool'
 'Fulham' 'Bournemouth' 'Wolves' 'Tottenham'] 20


In [12]:
from itertools import product

product(teams,teams)

In [14]:
teams_by_teams=list(product(teams,teams))

In [15]:
def drop_diag(cartprod):
    """
    Takes input cartesian product as list and removes duplicates
    """
    for el in cartprod:
        if el[0]==el[1]:
            cartprod.remove(el)
    return cartprod

In [18]:
fixtures=drop_diag(teams_by_teams)

In [77]:
sim_results=[]
for fix in fixtures:
    hometeam=fix[0]
    awayteam=fix[1]
    sim=predict_result(hometeam,awayteam,results,num_simulations=1)
    score=sim['FullSimulatedResults'][0]
    if score[0]>score[1]:
        res='HomeWin'
    elif score[0]<score[1]:
        res='AwayWin'
    else:
        res='Draw'
    sim_results.append({'HomeTeam':hometeam,'AwayTeam':awayteam,'Result':res,'Score':score,'HomeTeamGoals':score[0],'AwayTeamGoals':score[1]})

df_sim_results=pd.DataFrame(sim_results)
df_sim_results

,HomeTeam,AwayTeam,Result,Score,HomeTeamGoals,AwayTeamGoals
0,Southampton,Manchester United,AwayWin,"[0, 4]",0,4
1,Southampton,Leicester,AwayWin,"[1, 2]",1,2
2,Southampton,Leeds,HomeWin,"[6, 3]",6,3
3,Southampton,Everton,AwayWin,"[0, 3]",0,3
4,Southampton,Crystal Palace,AwayWin,"[1, 3]",1,3
...,...,...,...,...,...,...
375,Tottenham,Nottingham Forest,Draw,"[2, 2]",2,2
376,Tottenham,Liverpool,AwayWin,"[1, 3]",1,3
377,Tottenham,Fulham,HomeWin,"[2, 1]",2,1
378,Tottenham,Bournemouth,Draw,"[2, 2]",2,2


In [26]:
pip install pandasql

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for pandasql: filename=pandasql-0.7.3-py3-none-any.whl size=26800 sha256=ced98837474179306964ef96919e8f713237c7f4979198c80ea29f6e8b581554
  Stored in directory: c:\users\carlwhelan\appdata\local\pip\cache\wheels\68\5d\a5\edc271b998f909801d7956959f699b976cc9896075dc47c153
Successfully built pandasql
Note: you may need to restart the kernel to use updated packages.


In [27]:
from pandasql import sqldf

In [78]:
home_table_query="""
SELECT
HomeTeam AS Team,
COUNT(*) AS GP,
SUM(CASE
        WHEN Result='HomeWin'
        THEN 1
    END) AS W,
SUM(CASE
        WHEN Result='Draw'
        THEN 1
    END) AS D,
SUM(CASE
        WHEN Result='AwayWin'
        THEN 1
    END) AS L,
SUM(HomeTeamGoals) AS GF,
SUM(AWayTeamGoals) AS GA,
SUM(HomeTeamGoals) - SUM(AWayTeamGoals) AS GD,
SUM(CASE
        WHEN Result='HomeWin'
        THEN 3
        WHEN Result='Draw'
        THEN 1
    END) AS PTS
FROM

df_sim_results

GROUP BY HomeTeam
ORDER BY PTS desc
"""

home_table_df=sqldf(home_table_query)
home_table_df

,Team,GP,W,D,L,GF,GA,GD,PTS
0,Crystal Palace,19,14,2.0,3,49,25,24,44
1,Manchester City,19,13,3.0,3,50,30,20,42
2,Manchester United,19,12,5.0,2,45,22,23,41
3,Liverpool,19,12,5.0,2,52,26,26,41
4,Aston Villa,19,12,1.0,6,41,25,16,37
5,Wolves,19,10,3.0,6,42,39,3,33
6,Newcastle,19,10,2.0,7,47,43,4,32
7,West Ham,19,9,2.0,8,45,42,3,29
8,Everton,19,9,2.0,8,40,40,0,29
9,Brighton,19,9,2.0,8,37,36,1,29


In [79]:
away_table_query="""
SELECT
AwayTeam AS Team,
COUNT(*) AS GP,
SUM(CASE
        WHEN Result='AwayWin'
        THEN 1
    END) AS W,
SUM(CASE
        WHEN Result='Draw'
        THEN 1
    END) AS D,
SUM(CASE
        WHEN Result='HomeWin'
        THEN 1
    END) AS L,
SUM(AwayTeamGoals) AS GF,
SUM(HomeTeamGoals) AS GA,
SUM(AwayTeamGoals) - SUM(HomeTeamGoals) AS GD,
SUM(CASE
        WHEN Result='AwayWin'
        THEN 3
        WHEN Result='Draw'
        THEN 1
    END) AS PTS
FROM

df_sim_results

GROUP BY Team
ORDER BY PTS desc
"""

away_table_df=sqldf(away_table_query)
away_table_df

,Team,GP,W,D,L,GF,GA,GD,PTS
0,Manchester City,19,14,3.0,2,63,26,37,45
1,Tottenham,19,11,1.0,7,44,47,-3,34
2,Manchester United,19,11,NaN,8,41,28,13,33
3,Wolves,19,9,3.0,7,40,36,4,30
4,Crystal Palace,19,9,3.0,7,50,35,15,30
5,Chelsea,19,9,3.0,7,37,32,5,30
6,Bournemouth,19,8,5.0,6,31,30,1,29
7,Arsenal,19,7,8.0,4,54,42,12,29
8,Liverpool,19,8,3.0,8,50,39,11,27
9,Brighton,19,9,NaN,10,47,53,-6,27


In [80]:
(home_table_df.set_index('Team')+away_table_df.set_index('Team')).sort_values(by='PTS',ascending=False)

,GP,W,D,L,GF,GA,GD,PTS
Team,,,,,,,,
Manchester City,38,27,6.0,5,113,56,57,87
Crystal Palace,38,23,5.0,10,99,60,39,74
Manchester United,38,23,NaN,10,86,50,36,74
Liverpool,38,20,8.0,10,102,65,37,68
Aston Villa,38,20,4.0,14,69,54,15,64
Wolves,38,19,6.0,13,82,75,7,63
Tottenham,38,19,4.0,15,85,94,-9,61
Bournemouth,38,17,7.0,14,74,72,2,58
Newcastle,38,18,4.0,16,95,77,18,58
